In [ ]:
# ============================================================
# Setup do repositório
# ============================================================

import os

REPO_URL = "https://github.com/vongrossi/fazendo-um-llm-do-zero.git"
REPO_DIR = "fazendo-um-llm-do-zero"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}

os.chdir(REPO_DIR)

print("Diretório atual:", os.getcwd())


# Capítulo 02 — Texto vira número

Este notebook acompanha o Capítulo 02 da série **Fazendo um LLM do Zero**.

🎯 **Objetivos deste notebook:**
- Entender como texto vira números
- Construir um vocabulário a partir do zero
- Compreender o funcionamento da tokenização
- Criar pares input-target (janela deslizante)
- Preparar dados reais para um modelo GPT-like

Este notebook é o ponto onde a teoria começa a virar pipeline real.


## 1. Setup e Configuração

In [ ]:
!pip -q install -r 02-texto-vira-numero/requirements.txt


### 1.1 Imports e Seed

import sys
import numpy as np
import random
from collections import Counter

sys.path.append("02-texto-vira-numero")

from colab_setup import seed_everything

seed_everything(42)


## 2. Texto não é número

In [ ]:
text = """
LLMs aprendem padrões de linguagem.
LLMs precisam transformar texto em números.
Modelos só conseguem operar sobre vetores.
"""

print(text)


## 3. Tokenização Simples

In [ ]:
def simple_tokenize(text):
    return text.lower().split()

tokens = simple_tokenize(text)

print(tokens)


## 4. Construindo Vocabulário

In [ ]:
vocab = sorted(set(tokens))

token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for token, idx in token_to_id.items()}

print("Vocabulário:", vocab)


### 4.1 Encode

In [ ]:
def encode(text):
    return [token_to_id[token] for token in simple_tokenize(text)]

encoded = encode(text)
print(encoded)


### 4.2 Decode

In [ ]:
def decode(ids):
    return " ".join([id_to_token[i] for i in ids])

print(decode(encoded))


## 5. Introduzindo Embeddings

In [ ]:
# Criando embeddings simples

vocab_size = len(vocab)
embedding_dim = 4

embeddings = np.random.randn(vocab_size, embedding_dim)

print("Shape embeddings:", embeddings.shape)

In [2]:
# Obtendo embedding de tokens

token_ids = encode("llms aprendem padrões")

token_embeddings = embeddings[token_ids]

print(token_embeddings)

## 6. Sliding Window (ESSENCIAL)

In [2]:
# Função sliding window

def create_input_target_pairs(token_ids, context_size):

    inputs = []
    targets = []

    for i in range(len(token_ids) - context_size):
        input_seq = token_ids[i:i+context_size]
        target = token_ids[i+context_size]

        inputs.append(input_seq)
        targets.append(target)

    return np.array(inputs), np.array(targets)


### 6.1 Gerando pares input-target

In [ ]:
context_size = 3

inputs, targets = create_input_target_pairs(encoded, context_size)

print("Inputs:")
print(inputs)

print("\nTargets:")
print(targets)


## 7. Visualizando Pares

In [ ]:
# Mostrando pares de forma interpretável
for inp, tgt in zip(inputs, targets):

    print("INPUT :", decode(inp))
    print("TARGET:", id_to_token[tgt])
    print("---")

## 8. Preparando Entrada GPT-like

In [ ]:
# Convertendo inputs para embeddings
input_embeddings = embeddings[inputs]

print(input_embeddings.shape)

## 9. Intuição de Positional Embeddings

In [ ]:
# Simulação simples
positional_embeddings = np.random.randn(context_size, embedding_dim)

final_embeddings = input_embeddings + positional_embeddings

print(final_embeddings.shape)

## 10. Conclusão

### 10.1 O que acabamos de construir?

Criamos o pipeline básico que alimenta um GPT:

Texto → Tokens → IDs → Embeddings → Sequência contextual

Nos próximos capítulos vamos:
- aprender como atenção usa esses vetores
- construir blocos Transformer
- finalmente montar um GPT do zero

Este notebook representa a fundação matemática da linguagem em LLMs.


## 11. Extras (Experimentação)

1. Mude o context_size e observe os pares gerados.
2. Aumente embedding_dim e veja o impacto na matriz.
3. Teste outro dataset.
